<h1 style="text-align: center; vertical-align: middle;">Numerical Methods in Accelerator Physics</h1>
<h2 style="text-align: center; vertical-align: middle;">Python examples -- Week 10</h2>

<h2>Run this first!</h2>

Imports and modules:

In [ ]:
from config10 import (np, plt, tqdm, trange, c, epsilon_0,
                    beta, gamma, Machine, track_one_turn,
                    charge, mass, emittance, hamiltonian, U,
                    plot_hamiltonian, plot_rf_overview,
                    plot_dist)
%matplotlib inline

If the progress bar by `tqdm` (`trange`) in this document does not work, run this:

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

<h2>Example: Longitudinal Space Charge in CERN PS</h2>

Consider CERN PS for illustration once again:
- non-accelerating rf bucket at $V_{rf}=25$ kV
- injection energy $\gamma=3.13$
- bunch of $N_p=1\times 10^{12}$ particles
- rms bunch length of $\sigma_z=2$ m
- vacuum pipe of $\approx 10$ cm
- beam radius of $\approx 1$ cm

In [ ]:
m = Machine(gamma_ref=3.13, phi_s=0, voltage=25000)
#m = Machine(gamma_ref=7, phi_s=np.pi, voltage=25000)
# adjust here later!

The rf harmonic determines the width of an rf bucket:

In [ ]:
m.harmonic

Can you determine the location of the left and right unstable fix points, which limit the rf bucket?

In [ ]:
rfbucket_z_right = m.circumference/m.harmonic/2
rfbucket_z_left = -m.circumference/m.harmonic/2

In [ ]:
plot_rf_overview(m);

We initialize a Gaussian bunch distribution with small rms bunch length $\sigma_z=2$ m (so that the small-amplitude approximation applies):

In [ ]:
sigma_z = 2

The matched rms momentum spread $\sigma_{\Delta p}$ (remember, $\sigma_{\Delta p}$ and $\sigma_z$ are linked via equal Hamiltonian values, the equilibrium condition):

In [ ]:
sigma_deltap = np.sqrt(
    2 * m.p0() / np.abs(m.eta(0)) * 
    charge * m.voltage * np.pi * m.harmonic / (beta(gamma(m.p0())) * c * m.circumference**2)
) * sigma_z

In [ ]:
sigma_deltap / m.p0()

Number of simulated macro-particles (not too low such as to well resolve the line charge density of the bunch):

In [ ]:
N = 100000

Generation of macro-particles by initialising the phase-space distributio

In [ ]:
np.random.seed(12345)

z = np.random.normal(loc=0, scale=sigma_z, size=N)
deltap = np.random.normal(loc=0, scale=sigma_deltap, size=N)

In [ ]:
plot_hamiltonian(m, dpmax=0.005)
plt.scatter(z[::N // 1000], deltap[::N // 1000] / m.p0(), marker='.', s=1);

Next, we define a regular 1D grid for discretisation -- we will slice up the beam into bins (the slices):

In [ ]:
N_slices = 100

slice_boundaries = np.linspace(rfbucket_z_left, rfbucket_z_right, N_slices + 1, endpoint=True)

In [ ]:
slice_centres = (slice_boundaries[1:] + slice_boundaries[:-1]) / 2

And we are now in the position to compute a discretised line charge density $\lambda(z)$ from the $z$ distribution of the macro-particles:

In [ ]:
N_per_slice, slice_boundaries = np.histogram(z, bins=slice_boundaries)

In [ ]:
plt.step(slice_centres, N_per_slice, where='mid')
plt.xlabel('$z$ [m]')
plt.ylabel('Number of particles / slice');

The bunch has a total intensity of $N_p=1\times 10^{12}$ real particles. What total charge does a macro-particle carry?

In [ ]:
charge_per_mp = 1e12 / N * charge

In [ ]:
lmbda = charge_per_mp * N_per_slice / np.diff(slice_boundaries)

We evaluate the geometry factor $g$:

In [ ]:
r_pipe = 0.1
r_beam = 0.01

g = 1 + 2 * np.log(r_pipe / r_beam)

If the space-charge field is given by $E_z\propto \frac{d\lambda}{dz}$, the space-charge potential is given by the same expression but with $V_\mathrm{sc}\propto -\lambda$!

In [ ]:
V_sc = g / (4 * np.pi * epsilon_0 * m.gamma_ref**2) * lmbda

In [ ]:
plt.plot(slice_centres, V_sc)
plt.xlabel('$z$ [m]')
plt.ylabel('$|V_{sc}|$ [V]');

Compute the derivative of the line charge density via second-order (symmetric) finite difference,

$\lambda'(z)\approx\frac{\lambda(z+\Delta z)-\lambda(z-\Delta z)}{2\Delta z}$

In [ ]:
lmbda_prime = np.gradient(lmbda, slice_centres)

In [ ]:
plt.step(slice_centres, 1e9 * lmbda_prime)
plt.xlabel('$z$ [m]')
plt.ylabel("$\lambda'(z)$ [nC/m${}^2$]");

The longitudinal space-charge field is thus given by:

In [ ]:
E_z = -g / (4 * np.pi * epsilon_0 * m.gamma_ref**2) * lmbda_prime

<h3>Comparison with external phase focusing from rf</h3>
Compute the effective external RF field as a mean via the rf wave kick per circumference:

In [ ]:
E_rf = m.voltage / m.circumference * np.sin(
    -m.harmonic * slice_centres * 2 * np.pi / m.circumference + m.phi_s)
# E_rf = -np.gradient(U(slice_centres, m) / charge * c, slice_centres)

The external phase focusing by the rf wave and the space-charge field sum up:

In [ ]:
plt.plot(slice_centres, E_rf, c='k', label='RF')
plt.plot(slice_centres, E_z, c='r', label='SC')
plt.plot(slice_centres, E_rf + E_z, c='lightblue', label='RF+SC')

plt.legend()
plt.xlabel('$z$ [m]')
plt.ylabel('mean $E$ [V/m]');

<h2>Microwave instability</h2>
We implement the space-charge kick for the tracking:

In [ ]:
def space_charge_kick(z_n, deltap_n, machine, charge_per_mp=charge_per_mp, N_slices=N_slices, g=g):
    m = machine
    rfbucket_z_right = m.circumference / m.harmonic / 2
    rfbucket_z_left = -rfbucket_z_right
    
    # slicing
    slice_boundaries = np.linspace(rfbucket_z_left, rfbucket_z_right, N_slices + 1, endpoint=True)
    N_per_slice, _ = np.histogram(z_n, bins=slice_boundaries)
    # find slice index for each particle:
    slice_idx_per_mp = np.floor((z_n - slice_boundaries[0]) / 
                                (slice_boundaries[1] - slice_boundaries[0])).astype(int)
    
    # get line charge density derivative
    lmbda = charge_per_mp * N_per_slice / np.diff(slice_boundaries)
    lmbda_prime = np.gradient(lmbda, slice_boundaries[:-1])
    
    # space-charge field per slice
    E_z = -g / (4 * np.pi * epsilon_0 * m.gamma_ref**2) * lmbda_prime
    
    # momentum update
    deltap_n1 = deltap_n - charge * E_z[slice_idx_per_mp] * m.circumference / (beta(m.gamma_ref) * c)
    return deltap_n1

The kicks which the particles receive look as follows:

In [ ]:
plt.scatter(z, (space_charge_kick(z, deltap, m) - deltap) / m.p0())
plt.xlabel('$z$ [m]')
plt.ylabel('$(\Delta p_{n+1} - \Delta p_{n}) / p_0$');

We implement the one-turn tracking:

In [ ]:
def track_one_turn(z_n, deltap_n, machine, **kwargs):
    m = machine
    # half drift
    z_nhalf = z_n - m.eta(deltap_n) * deltap_n / m.p0() * m.circumference / 2
    # rf kick
    amplitude = charge * m.voltage / (beta(gamma(m.p0())) * c)
    phi = m.phi_s - m.harmonic * 2 * np.pi * z_nhalf / m.circumference
    
    m.update_gamma_ref()
    deltap_n1 = deltap_n + amplitude * (np.sin(phi) - np.sin(m.phi_s))
    # space-charge kick
    deltap_n1 = space_charge_kick(z, deltap_n1, m, **kwargs)
    # half drift
    z_n1 = z_nhalf - m.eta(deltap_n1) * deltap_n1 / m.p0() * m.circumference / 2
    return z_n1, deltap_n1

We simulate just above transition, continuing from the end of the previous part.

In [ ]:
assert m.gamma_ref > 1 / np.sqrt(m.alpha_c), 'Initialise the machine above transition energy!'

We simulate for a bit more than 1 synchrotron period above transition at 10x higher intensity than the initial nominal parameters:

In [ ]:
n_turns = 3000

intensity_factor = 10

... and record the longitudinal emittance:

In [ ]:
epsn_z = np.zeros(n_turns, dtype=np.float64)

epsn_z[0] = emittance(z, deltap)

... as well as the bunch profiles (just as we did in the tomography lecture 09):

In [ ]:
profiles = [np.histogram(z, bins=slice_boundaries)[0]]

Let's track! (May take a couple of minutes!)

In [ ]:
for i_turn in trange(1, n_turns):
    z, deltap = track_one_turn(z, deltap, m, 
                               charge_per_mp=charge_per_mp * intensity_factor, 
                               N_slices=400)
    
    # record:
    epsn_z[i_turn] = emittance(z, deltap)
    profiles += [np.histogram(z, bins=slice_boundaries)[0]]

Let us have a look at the final bunch profile:

In [ ]:
N_per_slice, slice_boundaries = np.histogram(z, bins=slice_boundaries)

plt.step(slice_centres, N_per_slice, where='mid')
plt.xlabel('$z$ [m]')
plt.ylabel('Number of particles / slice');

Let's plot the emittance evolution:

In [ ]:
plt.plot(np.arange(n_turns), 100 * (epsn_z - epsn_z[0]) / epsn_z[0])

plt.xlabel('Turns')
plt.ylabel('$\Delta \epsilon_z/\epsilon_{z0}$ [%]');

Look into phase space and a couple of particles:

In [ ]:
plot_hamiltonian(m, dpmax=0.005)
plt.scatter(z[::N // 1000], deltap[::N // 1000] / m.p0(), marker='.', s=1);

A density plot of the longitudinal phase space shows the damage:


In [ ]:
plt.hist2d(z, deltap/m.p0(), bins=100)

plt.xlabel('$z$ [m]')
plt.ylabel('$\delta$');

And the bunch profiles over time:

In [ ]:
plt.imshow(profiles, origin='lower', cmap='jet', extent=(rfbucket_z_left, rfbucket_z_right, 0, n_turns))
plt.gca().set_aspect(np.diff(plt.xlim()) / np.diff(plt.ylim()))
plt.xlabel('$z$ [m]')
plt.ylabel('Turn')
plt.colorbar(label='Density');